In [46]:
import pandas as pd
import requests
import logging
import time
import numpy as np

import folium
from folium import plugins

from IPython.core.display import HTML
pd.set_option('display.max_colwidth', -1)

In [79]:

import pyodbc 
server = '10.46.64.155' 
database = 'DRPZ_ReportsDB' 
username = 'Shiryaeva_EI' 
password = 'Rosbank11348' 

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [149]:
unit = 'STL'
src_stm_id = 4
bis_client_id = 'V09029'
stmt = "select *  from  sg_mart.dbo.odm_party p  where unit = '{}' and  bis_client_id = '{}' ".format(unit,bis_client_id)
# print(stmt)

client_info = pd.read_sql_query(stmt, cnxn)
display(HTML(client_info.T.to_html()))


,0
ODM_ID,1733583
SIEBEL_ID,1-28L7-6953
PARTY_TYPE_CD,Person
NAME,ЖАНЭ Виктор Олегович
NO_PHONE_FL,None
CLIENT_SEGMENT,AA
IS_OWN,РБ
GOLD_CLIENT_ID,None
UNIT,STL
BIS_CLIENT_ID,V09029


In [150]:

stmt = "select *  from  sg_mart.dbo.odm_agreement p  where bis_unit = '{}' and  con_bis_id = '{}' ".format(unit,bis_client_id)
# print(stmt)

credit_info = pd.read_sql_query(stmt, cnxn)
display(HTML(credit_info[["EXT_ID", "START_DT", "SUB_TYPE_CD", "STATUS_CD", "PROD_GROUP", "REAGE_DEBT_DAY", "DEBT_DAY", 
                          "VAR_SUM"]].T.to_html()))

,0
EXT_ID,9379V09029200
START_DT,2017-07-17 00:00:00.0000000
SUB_TYPE_CD,Credit
STATUS_CD,Open. Overdue
PROD_GROUP,потребительский
REAGE_DEBT_DAY,353
DEBT_DAY,353
VAR_SUM,496907


In [82]:
stmt = """SELECT dateadd(month, -1, dateadd(day,1,eomonth(CREATED))) AS REP_MONTH,
case when source = '3D Secure' then 'Автоматич загрузка'
when source_cd = 'BIS-SMS' then 'Автоматич загрузка'
when source ='SMS' then 'Автоматич загрузка'
when source = 'Mobile client Bank' then 'Автоматич загрузка' 
when source_cd = 'BIS-ATM' then 'Skip tracing'
when source_cd = 'MPZ+IFS' then 'Skip tracing'
when source = 'MPZ+IFS' then 'Skip tracing'
when source = 'Siebel' then 'Skip tracing'
when source = 'Transactions' then 'Skip tracing'
when source_cd = 'ITM400' and source = 'BIS' then 'Автоматич загрузка'
when source_cd = 'ITM400' and source = 'Siebel' then 'Skip tracing'
when source in ('BKI RSB', 'CND Company') then 'Пилот'
when source in ('NBKI') then 'БКИ'
else 'Other'
end as source_type,

case when source = '3D Secure' then '3D Secure'
when source_cd = 'BIS-SMS' then 'SMS-info'
when source ='SMS' then 'SMS-info'
when source_cd = 'BIS-ATM' then 'ATM-Host'
when source = 'Transactions' then 'Transactions (SMS)'
when source_cd = 'ITM400' and source = 'BIS' then 'BIS'
when source_cd = 'ITM400' and source = 'Siebel' then 'Вручную из Siebel'
when source in ('BKI RSB', 'CND Company') then source
when source = 'Mobile client Bank' then source 
when source in ('NBKI') then source
else coalesce(source_cd, source) 
end as source_subtype,
comm_string,
sum(case when ATTEMPT = 0 and cnt_new_phones_with_calls >0 then cnt_new_phones_with_calls else FLAG_attempt end) as cnt_calls_attempt,
COUNT(DISTINCT COMM_STRING) AS CNT_FOUND_PHONES,
SUM(FLAG_attempt) AS FLAG_attempt,
SUM(FLAG_comments) AS FLAG_comments,
SUM(ATTEMPT) AS CNT_ATTEMPT,
SUM(active_try) active_try,
SUM(excl_try) excl_try,
SUM(incorrect_try) incorrect_try,
SUM(cnt_new_phones_debtor) cnt_new_phones_debtor,
SUM(cnt_new_phones_third) cnt_new_phones_third,
SUM(cnt_new_phones_with_calls) cnt_new_phones_with_calls,
SUM(cnt_promises) cnt_promises,
SUM(PROMISE_KEPT) PROMISE_KEPT
FROM DRPZ_ReportsDB.LEA.SKIP_TRACING_RB_new t
where t.client_id = {}
GROUP BY dateadd(month, -1, dateadd(day,1,eomonth(CREATED))),
SOURCE_CD,
SOURCE, 
comm_string""".format(client_info["ODM_ID"][0])

phone_info = pd.read_sql_query(stmt, cnxn)
display(HTML(phone_info.T.to_html()))

,0,1,2,3
REP_MONTH,2018-06-01,2018-06-01,2018-07-01,2018-08-01
source_type,Skip tracing,Other,Skip tracing,Other
source_subtype,MPZ+IFS,ODS,MPZ+IFS,ODS
comm_string,8123119258,9818775966,9112938887,9522322615
cnt_calls_attempt,1,1,1,1
CNT_FOUND_PHONES,1,1,1,1
FLAG_attempt,0,1,1,1
FLAG_comments,1,1,1,1
CNT_ATTEMPT,0,5,5,3
active_try,0,1,1,1


In [84]:
stmt = """select t.*,  atm.[ID Банкомата]
from  [EXTERNAL_DB].[dbo].[atm_client_tr] t 
left join external_db.[atm].[export_atms] atm on t.ID_ATM like '%' + atm.[ID Банкомата] + '%'
where src_stm_id = {} and saan_cr = '{}'""".format(src_stm_id, bis_client_id) 

# stmt = """select t.* 
# from  [EXTERNAL_DB].[dbo].[atm_client_tr] t 
# where src_stm_id = {} and saan_cr = '{}'""".format(src_stm_id, bis_client_id) 


    
transactions_info = pd.read_sql_query(stmt, cnxn)
display(HTML(transactions_info[["JBPDP","ID_ATM", "Адрес строкой RU", "Место установки RU", "city_name"]].to_html()))    

,JBPDP,ID_ATM,Адрес строкой RU,Место установки RU,city_name
0,2018-03-02,4405030178656369 1 180302 140611 ATM ROSBANK ATM64110 ATM - Внесение наличных,Город Санкт-Петербург Проспект Просвещения Дом 34А,Отделение,Санкт-Петербург
1,2018-03-02,4405030178656369 1 180302 140512 ATM ROSBANK ATM64110 ATM - Внесение наличных,Город Санкт-Петербург Проспект Просвещения Дом 34А,Отделение,Санкт-Петербург
2,2018-02-05,4405030178656369 1 180205 144518 ATM ROSBANK ATM54471 ATM - Внесение наличных,"""Область Ленинградская Район Ломоносовский Село Виллозское сельское поселение """"Офицерское село"""" Дом Волхонское шоссе, квартал 1, д. 11, кор.3.""","""Склад ЗАО """"БИК""""""","""Область Ленинградская Район Ломоносовский"
3,2018-02-05,4405030178656369 1 180205 145440 ATM ROSBANK ATM54471 ATM - Внесение наличных,"""Область Ленинградская Район Ломоносовский Село Виллозское сельское поселение """"Офицерское село"""" Дом Волхонское шоссе, квартал 1, д. 11, кор.3.""","""Склад ЗАО """"БИК""""""","""Область Ленинградская Район Ломоносовский"
4,2018-02-05,4405030178656369 1 180205 144810 ATM ROSBANK ATM54471 ATM - Внесение наличных,"""Область Ленинградская Район Ломоносовский Село Виллозское сельское поселение """"Офицерское село"""" Дом Волхонское шоссе, квартал 1, д. 11, кор.3.""","""Склад ЗАО """"БИК""""""","""Область Ленинградская Район Ломоносовский"


In [101]:
atm_coord = pd.read_csv(r'atm_upd.txt')
atm_coord.tail()

,Unnamed: 0,ID Банкомата,Адрес строкой RU,coord,coord_x,0,1
1604,1684,ATM71943,Край Приморский Город Фокино Поселок городского типа Дунай Улица Ленина Дом 30,"<[OK] Yandex - Geocode [Russia, Primorye Territory, Urban-Type Settlement of Dunay, ulitsa Lenina, 30]>","['42.882147', '132.362257']",42.882147,132.362257
1605,1685,ATM71948,Республика Марий Эл Город Йошкар-Ола Улица Карла Либкнехта Дом 55,"<[OK] Yandex - Geocode [Russia, Republic of Mari El, Yoshkar-Ola, Karla Libknekhta Street, 55]>","['56.63857', '47.952016']",56.638570,47.952016
1606,1686,ATM71949,Город Санкт-Петербург Набережная Грибоедова Канала Дом 13,"<[OK] Yandex - Geocode [Russia, Saint Petersburg, Kanala Griboedova Embankment, 13]>","['59.937039', '30.326405']",59.937039,30.326405
1607,1687,ATM64110,NaN,NaN,Город Санкт-Петербург Проспект Просвещения Дом 34А,60.053620,30.334257
1608,1688,ATM54471,NaN,NaN,"Область Ленинградская Район Ломоносовский Село Виллозское сельское поселение ""Офицерское село"" Дом Волхонское шоссе, квартал 1, д. 11, кор.3.",59.695032,30.107739


In [104]:
# transactions_info.join(atm_coord, )
transactions_info_coord = pd.merge(transactions_info, atm_coord, on='ID Банкомата')

In [106]:
lat_center =  np.mean([transactions_info_coord["0"].min(), transactions_info_coord["0"].max()])
long_center =  np.mean([transactions_info_coord["1"].min(), transactions_info_coord["1"].max()])


locations = transactions_info_coord[["0","1"]]
locationlist = locations.values.tolist()
times = transactions_info_coord["JBPDP"]

0    2018-03-02
1    2018-03-02
2    2018-02-05
3    2018-02-05
4    2018-02-05
Name: JBPDP, dtype: object

In [151]:
map_track = folium.Map(
    location = [ lat_center, long_center]   ,
    zoom_start = 10
)

plugins.MarkerCluster(locationlist, popups = times).add_to(map_track) 
map_track

In [141]:


stmt = "select * from external_db.[mfm].[rosbank_19_col_m12] where odm_id = '{}' ".format(client_info["ODM_ID"][0])
# print(stmt)

mfm_12 = pd.read_sql_query(stmt, cnxn)
display(HTML(mfm_12.T[mfm_12.T.iloc[:,0].notnull()].to_html()))





,0
ODM_ID,1733583
UNIT_NAME,STL
COMM_STRING,7.98188e+10
APPLICATION_ID,79818775966_2019-01-21
MAXREMAINMAX,0
MAXREMAINSTDDEV,0
MTTRANSAVG,9040
MTTRANSMIN,9040
MTTRANSMAX,9040
MTTRANSMED,9040


In [143]:


stmt = "select * from external_db.[mfm].[rosbank_19_col_m6] where odm_id = '{}' ".format(client_info["ODM_ID"][0])
# print(stmt)

mfm_6 = pd.read_sql_query(stmt, cnxn)
display(HTML(mfm_6.T[mfm_6.T.iloc[:,0].notnull()].to_html()))





,0
ODM_ID,1733583
UNIT_NAME,STL
COMM_STRING,7.98188e+10
APPLICATION_ID,79818775966_2019-01-21


In [145]:


stmt = "select * from external_db.[mfm].[rosbank_19_col_m3] where odm_id = '{}' ".format(client_info["ODM_ID"][0])
# print(stmt)

mfm_3 = pd.read_sql_query(stmt, cnxn)
display(HTML(mfm_3.T[mfm_3.T.iloc[:,0].notnull()].to_html()))





,0
ODM_ID,1733583
UNIT_NAME,STL
COMM_STRING,7.98188e+10
APPLICATION_ID,79818775966_2019-01-21
load_date,2019-02-11 17:40:34.257000


In [148]:
stmt = """select ilma.*, agr.ext_id  
from external_db.ilma.input_data ilma 
	inner join sg_mart.dbo.odm_agreement agr on agr.ext_id = ilma.ac13 and agr.start_dt = ilma.ptdtn
where  agr.bis_unit = '{}' and  agr.con_bis_id = '{}' """.format(unit,bis_client_id)
# print(stmt)

ilma_info = pd.read_sql_query(stmt, cnxn)
display(HTML(ilma_info.T.to_html()))

""
ID
AGREEMENT_ID
AC13
PTDTN
ZALOG_ID
CAR_MARKMODEL
CAR_PTS_NUM
CAR_PTS_DT
CAR_REGTYPE
CAR_OB_DVIG


In [103]:
atm_coord[atm_coord["ID Банкомата"] == "ATM64110"]

,Unnamed: 0,ID Банкомата,Адрес строкой RU,coord,coord_x,0,1
1607,1687,ATM64110,NaN,NaN,Город Санкт-Петербург Проспект Просвещения Дом 34А,60.05362,30.334257


In [92]:
transactions_info[transactions_info["ID Банкомата"] == "ATM64110"]

,JBPDP,JBPAD,JBPAC,SRC_STM_ID,SAAB_CR,SAAN_CR,SAAS_CR,ID_ATM,Бизнес территория,Адрес строкой RU,...,ptdtn,prod_type,address_reg,address_fact,address_fact2,DELAY_DAYS,ptnam,ptbsd,idat,ID Банкомата
0,2018-03-02,47422810294990000171,40817810393770006125,4,9377,V09029,001,4405030178656369 1 180302 140611 ATM ROSBANK ATM64110 ATM - Внесение наличных,Санкт-Петербургский,Город Санкт-Петербург Проспект Просвещения Дом 34А,...,2017-07-17,Consumer,"197760,САНКТ-ПЕТЕРБУРГ Г,,КРОНШТАДТ Г,,,ЛЕНИНА ПР-КТ,,32,,10","197760,Санкт-Петербург г,,Кронштадт г,,Ленина пр-кт,32,,10","197760,САНКТ-ПЕТЕРБУРГ Г,,КРОНШТАДТ Г,,,ЛЕНИНА ПР-КТ,,32,,10",268,ЖАНЭ Виктор Олегович,1986-11-22,2018-12-26,ATM64110
1,2018-03-02,47422810294990000171,40817810393770006125,4,9377,V09029,001,4405030178656369 1 180302 140512 ATM ROSBANK ATM64110 ATM - Внесение наличных,Санкт-Петербургский,Город Санкт-Петербург Проспект Просвещения Дом 34А,...,2017-07-17,Consumer,"197760,САНКТ-ПЕТЕРБУРГ Г,,КРОНШТАДТ Г,,,ЛЕНИНА ПР-КТ,,32,,10","197760,Санкт-Петербург г,,Кронштадт г,,Ленина пр-кт,32,,10","197760,САНКТ-ПЕТЕРБУРГ Г,,КРОНШТАДТ Г,,,ЛЕНИНА ПР-КТ,,32,,10",268,ЖАНЭ Виктор Олегович,1986-11-22,2018-12-26,ATM64110


In [90]:
transactions_info["ID Банкомата"]

0    ATM64110
1    ATM64110
2    ATM54471
3    ATM54471
4    ATM54471
Name: ID Банкомата, dtype: object